In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import rpy2.robjects as robjects
import rpy2.robjects.pandas2ri as pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr, isinstalled
import rpy2.robjects.packages as rpackages
from rpy2.robjects import r
import sys

import sys
sys.path.insert(0, '/Users/alanma/Documents/CFA_python')
import faircause.faircause as faircause


In [2]:
# Ensure faircause is installed (either using conda or R)

print("Is faircause installed now?", isinstalled('faircause'))


Is faircause installed now? True


In [3]:
# import
base = importr('base')
faircause = importr('faircause')

# Load census dataset
data = robjects.r('''
    data("gov_census", package = "faircause")
    gov_census[seq_len(20000), ]  # Take first 20000 rows as in the vignette
''')

# Rename columns to match the vignette
data.columns = ['sex', 'age', 'race', 'hispanic_origin', 'citizenship', 'nativity',
                'marital', 'family_size', 'children', 'education_level', 'english_level',
                'salary', 'hours_worked', 'weeks_worked', 'occupation', 'industry',
                'economic_region']

# Convert to pd
with localconverter(robjects.default_converter + pandas2ri.converter):
  data = robjects.conversion.rpy2py(data)
data.reset_index(drop=True, inplace=True)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

In [4]:
data

,sex,age,race,hispanic_origin,citizenship,nativity,marital,family_size,children,education_level,english_level,salary,hours_worked,weeks_worked,occupation,industry,economic_region
0,male,64.0,black,no,1.0,native,married,2.0,0.0,20.0,0.0,43000.0,56.0,49.0,13-1081,928P,Southeast
1,female,54.0,white,no,1.0,native,married,3.0,1.0,20.0,0.0,45000.0,42.0,49.0,29-2061,6231,Southeast
2,male,38.0,black,no,1.0,native,married,3.0,1.0,24.0,0.0,99000.0,50.0,49.0,25-1000,611M1,Southeast
3,female,41.0,asian,no,1.0,native,married,3.0,1.0,24.0,0.0,63000.0,50.0,49.0,25-1000,611M1,Southeast
4,female,40.0,white,no,1.0,native,married,4.0,2.0,21.0,0.0,45200.0,40.0,49.0,27-1010,611M1,Southeast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,female,54.0,white,no,1.0,native,married,2.0,0.0,22.0,0.0,38000.0,40.0,39.0,27-3092,923,Far West
19996,male,43.0,other,yes,4.0,foreign-born,divorced,4.0,0.0,21.0,1.0,100000.0,40.0,49.0,33-3021,92MP,Far West
19997,male,35.0,AIAN,yes,4.0,foreign-born,married,4.0,2.0,22.0,2.0,73000.0,40.0,47.0,25-2030,6111,Far West
19998,male,26.0,white,no,1.0,native,married,2.0,0.0,18.0,0.0,60000.0,75.0,49.0,55-2010,928110P4,Far West


In [5]:
# placing columns
X = "sex"
W = ["marital", "family_size", "children", "education_level",
    "english_level", "hours_worked", "weeks_worked", "occupation",
    "industry"]
Z = ["age", "race", "hispanic_origin", "citizenship", "nativity",
    "economic_region"]
Y = "salary"
x0 = "male"
x1 = "female"


In [6]:
import faircause.faircause as faircause

fc_census = faircause.FairCause(data, X, Z, W, Y, x0, x1)

In [7]:
print(fc_census)

faircause object:

Attribute:       sex
Outcome:         salary
Confounders:     marital, family_size, children, education_level, english_level, hours_worked, weeks_worked, occupation, industry
Mediators:       age, race, hispanic_origin, citizenship, nativity, economic_region


In [8]:
fc_census.estimate_effects()

0         43000.0
1         45000.0
2         99000.0
3         63000.0
4         45200.0
           ...   
19995     38000.0
19996    100000.0
19997     73000.0
19998     60000.0
19999     56000.0
Name: salary, Length: 20000, dtype: float64
{'all': array([ 6037,  1188, 17537, ..., 13288, 19100, 16853], shape=(20000,)), 'id0': array([], shape=(0, 17), dtype=int64), 'id1': array([], shape=(0, 17), dtype=int64)}


NameError: name 'quit' is not defined

In [9]:
fc_census.summary()

faircause object summary:

Protected attribute:                 sex
Protected attribute levels:          male, female
Total Variation (TV): nan

TV decomposition(s):

TV_malefemale(y) (nan) = CtfDE_malefemale(y | male) (nan) - CtfIE_femalemale(y | male) (nan) - CtfSE_femalemale(y) (nan)


,measure,value,sd
0,ctfde,NaN,NaN
1,ctfie,NaN,NaN
2,ctfse,NaN,NaN
3,ett,NaN,NaN
4,expse_x0,NaN,NaN
5,expse_x1,6763.951937,716.052037
6,nde,-1082.300434,2833.913985
7,nie,13684.326491,2890.878019
8,te,-14766.626926,1067.002425
9,tv,NaN,NaN
